# `polars_bloomberg` Examples
(c) 2024 Marek Ozana

In [1]:
# Imports
from polars_bloomberg import BQuery
from datetime import date

## Bloomberg Data Point
```python
bdp(securities: List[str],
    fields: List[str],
    overrides: Optional[Sequence] = None,
    options: Optional[Dict] = None
)
```

In [2]:
# Bloomberg Data Point
with BQuery() as bq:
    df = bq.bdp(["SPX Index", "VIX Index"], ["NAME", "PX_LAST"])
df

security,NAME,PX_LAST
str,str,f64
"""SPX Index""","""S&P 500 INDEX""",6079.69
"""VIX Index""","""Cboe Volatility Index""",13.43


## Bloomberg Data History
```python
bdh(securities: List[str],
    fields: List[str],
    start_date: date,
    end_date: date,
    overrides: Optional[Sequence] = None,
    options: Optional[Dict] = None
)

```

In [3]:
# Bloomberg Data History
with BQuery() as bq:
    df = bq.bdh(
        ["SPY US Equity", "TLT US Equity"],
        ["PX_LAST", "VOLUME"],
        start_date=date(2019, 1, 1),
        end_date=date(2019, 1, 10),
        options={"adjustmentSplit": True},
    )
df


security,date,PX_LAST,VOLUME
str,date,f64,f64
"""SPY US Equity""",2019-01-02,250.18,1.26925199e8
"""SPY US Equity""",2019-01-03,244.21,1.44140692e8
"""SPY US Equity""",2019-01-04,252.39,1.42628834e8
"""SPY US Equity""",2019-01-07,254.38,1.031391e8
"""SPY US Equity""",2019-01-08,256.77,1.02512587e8
…,…,…,…
"""TLT US Equity""",2019-01-04,122.11,1.2970226e7
"""TLT US Equity""",2019-01-07,121.75,8.498104e6
"""TLT US Equity""",2019-01-08,121.43,7.737103e6


## Bloomberg Query Language
```python
bql(expression: str)
```

In [4]:
# Example: average PE ratio per sector
query = """
let(#avg_pe=avg(group(pe_ratio(), gics_sector_name()));)
get(#avg_pe) 
for(members('SPX Index'))
"""
with BQuery() as bq:
    df = bq.bql(query)
df

ID,#avg_pe,#avg_pe.REVISION_DATE,#avg_pe.AS_OF_DATE,#avg_pe.PERIOD_END_DATE,#avg_pe.ORIG_IDS,#avg_pe.GICS_SECTOR_NAME()
str,f64,date,date,date,str,str
"""Communication Services""",26.664228,2024-11-14,2024-12-04,2024-09-30,null,"""Communication Services"""
"""Consumer Discretionary""",25.798281,2024-11-27,2024-12-04,2024-11-02,null,"""Consumer Discretionary"""
"""Consumer Staples""",19.562332,2024-12-04,2024-12-04,2024-11-02,null,"""Consumer Staples"""
"""Energy""",19.171636,2024-11-12,2024-12-04,2024-09-30,null,"""Energy"""
"""Financials""",38.011491,2024-11-26,2024-12-04,2024-09-30,null,"""Financials"""
…,…,…,…,…,…,…
"""Industrials""",38.41907,2024-11-26,2024-12-04,2024-10-31,null,"""Industrials"""
"""Information Technology""",57.617628,2024-12-03,2024-12-04,2024-11-02,null,"""Information Technology"""
"""Materials""",25.683113,2024-11-21,2024-12-04,2024-09-30,null,"""Materials"""


In [5]:
# Example: Duration and ZSpread for search results
query="""
let(#dur=duration(duration_type=MODIFIED); 
    #zsprd=spread(spread_type=Z);) 
get(name(), #dur, #zsprd) 
for(screenresults(type=SRCH, screen_name='@COCO'))
"""

with BQuery() as bq:
    df = bq.bql(query)
df

ID,name(),#dur,#dur.DATE,#zsprd,#zsprd.DATE
str,str,f64,date,f64,date
"""YT389519 Corp""","""AKYATI 48 02/11/25""",null,2024-12-04,null,2024-12-04
"""ZO366943 Corp""","""SBIIN 7.74 PERP""",null,2024-12-04,null,2024-12-04
"""ZO417246 Corp""","""NAERIN Float PERP""",null,2024-12-04,null,2024-12-04
"""ZO455769 Corp""","""CBKIN 8.3 PERP""",null,2024-12-04,null,2024-12-04
"""ZO684719 Corp""","""NORC 6 ½ PERP""",null,2024-12-04,null,2024-12-04
…,…,…,…,…,…
"""ZN373952 Corp""","""SOCGEN 9 ⅜ PERP""",2.921201,2024-12-04,415.14048,2024-12-04
"""ZN448028 Corp""","""MINGNO Float PERP""",0.222819,2024-12-04,254.114,2024-12-04
"""ZN449141 Corp""","""MINGNO 7.12 PERP""",2.603761,2024-12-04,253.508096,2024-12-04
